# ♟️ Chess DQN - Google Colab Host

Run this notebook to host the Chess DQN backend on a free Google Colab GPU instance.

### Instructions:
1. **Upload Files**: Drag and drop your backend files into the file explorer on the left.
2. **Add Ngrok Token**: Create a free account at [ngrok.com](https://ngrok.com), get your authtoken, and verify it below.
3. **Run All Cells**: Click 'Runtime' > 'Run all'.

In [ ]:
# @title 0. Safety Check
import os
try:
    %cd /content
    print(f"Current working directory: {os.getcwd()}")
except:
    print("Error: Could not find /content. Please go to 'Runtime' > 'Restart session'")

In [ ]:
# @title 1. Install Dependencies
# Note: Pinned uvicorn version to avoid 'loop_factory' error with nest_asyncio
!pip install fastapi uvicorn==0.24.0.post1 python-chess pyngrok nest-asyncio

In [ ]:
# @title 2. Setup Ngrok Tunnel
from pyngrok import ngrok
import nest_asyncio
import os
import time

# ---------------------------------------------------------
# REPLACE THIS WITH YOUR AUTH TOKEN FROM dashboard.ngrok.com
NGROK_AUTH_TOKEN = "YOUR_NGROK_AUTH_TOKEN_HERE"
# ---------------------------------------------------------

# FORCE KILL any existing ngrok processes
os.system("pkill ngrok")
ngrok.kill()
time.sleep(1) # Give it a second to die

ngrok.set_auth_token(NGROK_AUTH_TOKEN)
nest_asyncio.apply()

In [ ]:
# @title 3. Start Backend Server
import uvicorn
import os
import sys
import nest_asyncio
import time

nest_asyncio.apply()

# Create tunnel to port 8000
try:
    # Retry connection logic
    public_url = ngrok.connect(8000).public_url
    print(f"\n🔥🔥🔥 YOUR API URL IS: {public_url} 🔥🔥🔥\n")
except Exception as e:
    print(f"❌ NGROK ERROR: {e}")
    print("Trying to kill and restart...")
    os.system("pkill ngrok")
    ngrok.kill()
    time.sleep(2)
    try:
        public_url = ngrok.connect(8000).public_url
        print(f"\n🔥🔥🔥 YOUR API URL IS: {public_url} 🔥🔥🔥\n")
    except Exception as e2:
        print(f"Failed again: {e2}")
        print("Please check your dashboard.ngrok.com to see if an agent is stuck online.")

if __name__ == "__main__":
    if not os.path.exists('main.py'):
        print("❌ ERROR: main.py not found in /content! Did you upload your files?")
    else:
        # FIX 1: Add current dir to sys.path explicitly
        sys.path.append(os.getcwd())
        
        try:
            # FIX 2: Import app directly to verify dependencies before uvicorn starts
            from main import app
            
            # Running manual server with APP INSTANCE (not string)
            config = uvicorn.Config(app, host="0.0.0.0", port=8000, log_level="info")
            server = uvicorn.Server(config)
            server.run()
        except ImportError as e:
            print(f"\n❌ FATAL IMPORT ERROR: {e}")
            print("This usually means a dependency is missing or a file wasn't uploaded.")
            print("Check that you uploaded: chess_env.py, trainer.py, smart_simulation.py, etc.")
        except Exception as e:
            print(f"\n❌ RUNTIME ERROR: {e}")